In [ ]:
pip install biopython

In [ ]:
# Aubrey Morales Figueroa
# BIOL-51000 Week 5

import numpy as np
import math
from Bio import PDB
from Bio.PDB import PDBParser
from matplotlib import pyplot
from numpy import array, dot, exp, identity, linalg, ones, sqrt, zeros, cross
from math import sqrt, cos, sin, acos, degrees

ATOMIC_NUMS = {'H':1,'C':12,'N':14,'O':16,'P':31,'S':32}

# Class representing a molecular structure.
class Structure:
    def __init__(self, name, conformation=0, pdbId=None):
        if not name:
            raise Exception( 'name must be set to non-empty string' )
        self._name = name
        self.conformation = conformation
        self.pdbId = pdbId
        self.chains = []

    # Method to delete specific chains from the structure.
    def delete(self):
        for chain in self.chains:
            chain.delete( )

    # Method to retrieve specific chains from the structure.
    def getChain(self, code):
        for chain in self.chains:
            if chain.code == code:
                return chain

        return None

    # Method to retrieve the pdbIDs.
    def getPdbIds(self):
        return list(self.pdbIds)

    # Method to retrieve the mass of the structure.
    def getMass(self):
        if hasattr(self, 'mass'):
            return self.mass

        mass = 0
        for chain in self.chains:
            mass += chain.getMass( )
        self.mass = mass
        return mass

    def getName(self):
        return self._name

    def setName(self, name):
        if not name:
            raise Exception('name must be set to non-empty string')
        self._name = name

    name = property(getName, setName)

# Class representing a chain within a molecular structure.
class Chain:
    def __init__(self, structure, chainId):
        self.structure = structure
        self.chainId = chainId
        self.residues = []
        self.resDict = {}

    # Method to add a residue to the chain.
    def addResidue(self, residue):
        self.residues.append(residue)
        self.resDict[residue.seqId] = residue

    # Method to retrieve a specific residue from the chain.
    def getResidue(self, seqId):
        return self.resDict.get(seqId)

# Class representing a residue, within a chain, within a molecular structure.
class Residue:
    def __init__(self, chain, seqId, code):
        self.chain = chain
        self.seqId = seqId
        self.code = code
        self.atoms = []
        self.atomDict = {}

    # Method to add an atom to the reside.
    def addAtom(self, atom):
        self.atoms.append(atom)
        self.atomDict[atom.name] = atom

     # Method to retieve a specific atom from the reside.
    def getAtom(self, name):
        return self.atomDict.get(name)

# Class representing an atom within a residue, within a chain, within a molecular structure.
class Atom:
    def __init__(self, residue, name, coords, element):
        self.residue = residue
        self.name = name
        self.coords = coords
        self.element = element

        # Check if atom exists in residue.
        if any(a.name == name for a in residue.atoms):
           raise Exception(f'Atom name="{name}" already used in residue {residue.seqId} ({residue.code})')

        residue.atoms.append(self)

    # Method to delete atom from residue.
    def delete(self):
        del self.residue.atomDict[self.name]
        self.residue.atoms.remove(self)

# Function to retrieve a molecular structure from a PDB file.
def getStructuresFromFile(fileName):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', fileName)

    strucObjs = []
    for model in structure:
        for chain in model:
            chainObj = Chain(structure, chain.id)
            pdb_id = structure.id

            found = False
            for struc in strucObjs:
                if struc.pdbId == pdb_id:
                    struc.chains.append(chainObj)
                    found = True
                    break

            if not found:
                new_struc = Structure("SomeName", pdbId=pdb_id)
                new_struc.chains.append(chainObj)
                strucObjs.append(new_struc)

            for residue in chain:
                resObj = Residue(chainObj, residue.id[1], residue.id[0])
                for atom in residue:
                    atomObj = Atom(resObj, atom.id, atom.coord, atom.element)
                    resObj.addAtom(atomObj)
                chainObj.addResidue(resObj)

    return strucObjs

# Function to guess the molecule type of a residue (DNA, RNA, or protein), based on atom names.
def guessResidueMolType(residue):
    if residue.getAtom("CA") and residue.getAtom("N"):
            return 'protein'

    elif residue.getAtom("C5'") and residue.getAtom("C3'"):

          if residue.getAtom("02'"):
                return 'RNA'
          else:
                return 'DNA'

    return 'other'

# Function to retieve atom coordinates from molecular structure.
def getAtomCoords(structure):
    coords = []
    atoms = []

    for chain in structure.chains:
        for residue in chain.residues:
            for atom in residue.atoms:
                coords.append(atom.coords)
                atoms.append(atom)

    return atoms, array(coords)

# Function to rotate the structures.
def rotateStructureNumPy(structure, axis=array([1, 0, 0]), angle=0):
    rMatrix = array(getRotationMatrix(axis, angle))

    atoms, coords = getAtomCoords(structure)

    coords = dot(coords, rMatrix.T)

    for index, atom in enumerate(atoms):
        atom.coords = list(coords[index])

# Function to retrieve the rotation matrix.
def getRotationMatrix(axis, angle):
  vLen = math.sqrt( sum([xyz*xyz for xyz in axis]) )
  x, y, z = [xyz/vLen for xyz in axis]
  c = math.cos(angle)
  d = 1-c
  s = math.sin(angle)
  R = [[c+d*x*x,   d*x*y-s*z, d*x*z+s*y],
       [d*y*x+s*z, c+d*y*y,   d*y*z-s*x],
       [d*z*x-s*y, d*z*y+s*x, c+d*z*z  ]]
  return R

# Function to center atom coordinates.
def centerCoords(coords, weights):
    wCoords = coords.transpose( ) * weights

    xyzTotals = wCoords.sum(axis = 1)

    center = xyzTotals/sum(weights)

    coords -= center

    return coords, center

# Function to align coordinates between two structures.
def alignCoords(coordsA, coordsB, weights = None):
    n = len(coordsA)
    if weights is None:
        weights = ones(n)

    rMat = dot(coordsB.transpose() * weights, coordsA)

    rMat1, scales, rMat2 = linalg.svd(rMat)

    sign = linalg.det(rMat1) * linalg.det(rMat2)

    if sign < 0:
        rMat1[:, 2] *= -1

    rotation = dot(rMat1, rMat2)

    coordsB = dot(coordsB, rotation)

    return rotation, coordsB

# Function to calculate the Root-Mean-Square-Deviations between coordinates.
def calcRmsds(refCoords, allCoords, weights):
    rmsds = []
    totalWeight = sum(weights)
    totalSquares = zeros(refCoords.shape)

    for coords in allCoords:
        delta = coords - refCoords
        squares = delta * delta
        totalSquares += squares
        sumSquares = weights * squares.sum(axis=1)
        print(sumSquares)
        rmsd = sqrt(sum(sumSquares) / totalWeight)
        rmsds.append(rmsd)

    nStruct = len(allCoords)
    atomRmsds = sqrt(totalSquares.sum() / (nStruct * refCoords.shape[0]))

    return rmsds, atomRmsds

# Function to align coordinates by superimposing them onto a reference coordinate set.
def superimposeCoords(allCoords, weights, threshold = 5.0):
    nStruct = len(allCoords)

    refCoords = allCoords[0]
    meanCoords = zeros(refCoords.shape)
    rotations = []

    for index, coords in enumerate(allCoords):
        if index == 0:
            rotation = identity(3)
        else:
            rotation, coords = alignCoords(refCoords, coords, weights)
            allCoords[index] = coords

        rotations.append(rotation)
        meanCoords += coords

    meanCoords /= nStruct

    rmsds, atomRmsds = calcRmsds(meanCoords, allCoords, weights)

    bestRmsd = min(rmsds)
    bestIndex = rmsds.index(bestRmsd)
    bestCoords = allCoords[bestIndex]

    weightScale = atomRmsds / threshold
    weights = weights.astype(float)
    weights *= exp(-weightScale * weightScale)

    meanCoords = bestCoords.copy()

    for index, coords in enumerate(allCoords):
        if index != bestIndex:
            rotation, coords = alignCoords(bestCoords, coords, weights)
            rotations[index] = rotation
            allCoords[index] = coords
            meanCoords += coords

    meanCoords /= nStruct
    weights = ones(len(weights))
    rmsds, atomRmsds = calcRmsds(meanCoords, allCoords, weights)

    return allCoords, rmsds, atomRmsds, rotations

# Function to align molecular structures by aligning their atom coordinates.
def superimposeStructures(structures):
    weights = None
    allCoords = []

    for structure in structures:
        atoms, coords = getAtomCoords(structure)

        if weights is None:
            weights = [ATOMIC_NUMS[atom.element] for atom in atoms]
            weights = array(weights)

        coords, center = centerCoords(coords, weights)
        allCoords.append(coords)

    results = superimposeCoords(allCoords, weights)
    allCoords, rmsds, atomRmsds, rotations = results

    for i, structure in enumerate(structures):
        atoms, oldCoords = getAtomCoords(structure)

        for j, atom in enumerate(atoms):
            atom.coords = allCoords[i][j]

    return rmsds, atomRmsds

# Run
if __name__ == '__main__':
    fileName = '1UST.pdb'

    strucA = getStructuresFromFile(fileName)[0]
    strucB = getStructuresFromFile(fileName)[0]

    rotateStructureNumPy(strucA,(1,0,0),1)

    print("The sum of the square values along the spatial axis:")
    print()

    rmsds, atomRmsds = superimposeStructures([strucA, strucB])

    print()
    print("The RMSDs:")
    print(rmsds)

#EOF

The sum of the square values along the spatial axis:

[2.75398684e-12 2.75398684e-12 2.35592336e-12 ... 2.05567474e-13
 2.12544201e-13 2.12544201e-13]
[2.75398684e-12 2.75398684e-12 2.35592340e-12 ... 2.05567474e-13
 2.12544204e-13 2.12544204e-13]
[1.96713346e-13 1.96713346e-13 1.96326948e-13 ... 2.05567473e-13
 2.12544201e-13 2.12544201e-13]
[1.96713346e-13 1.96713346e-13 1.96326951e-13 ... 2.05567473e-13
 2.12544204e-13 2.12544204e-13]

The RMSDs:
[1.97533874542178e-07, 1.9753387456093888e-07]
